In [ ]:
#%matplotlib inline  # inline is a picture only.

import matplotlib
#matplotlib.use("WxAgg")
matplotlib.use("TkAgg")  #for pop up window
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import os
import seaborn as sns

os.getcwd()
# TO get a boxplot to see the number of 4-Rooms flats available for Nov 2018 Sale of balance flats

unitDetails = np.genfromtxt("Merged_unitsDetails_DistToMrt.csv", delimiter=",",skip_header=1,
                dtype=[('town','U24'),('rmType','U7'),('blk','U6'),('unitNum','U10'),
                      ('level','i4'),('sqm','i8'),
                      ('sellingPrice','i8'),('color','U12') ,('repurchasedFlat','U30'),('leaseLessThan60Yrs','U10'),
                      ('street','U30'),('probableCompletionDate','U10'),('deliveryPossessionDate','U30'),
                      ('leaseCommencementDate','U15'),('availableEthnicQuotaMl','U20'),
                      ('availableEthnicQuotaCh','U25'),('availableEthnicQuotaIn','U20'),('lastUpdate','U60'),
                      ('NearestMrt','U30'),('DistToMrt','f8'),('blkX','i8'),('blkY','i8')],
                missing_values=['na','-',''],filling_values=0, comments='!')

labels = ['Ang Mo Kio','Bedok','Bukit Batok','Bukit Merah','Bukit Panjang','Choa Chu Kang',
                'Clementi','Geylang','Hougang','Jurong East','Jurong West','Kallang / Whampoa','Pasir Ris','Punggol',
                'Queenstown','Sembawang','Sengkang','Serangoon','Tampines','Toa Payoh', 'Woodlands','Yishun']
# defining names
DistToMrt= unitDetails['DistToMrt']
town = unitDetails['town']
rmType= unitDetails['rmType']
sellingPrice=unitDetails['sellingPrice']
floorlevel = unitDetails['level']

#print(town)
#print(DistToMrt)
townToMrt= zip(town,DistToMrt)
townToMrt =set(townToMrt)

# Gathering data for each line on distance
#for i in labels:
#    line_i=[]
#    for name in townToMrt:
#        if name[0]== i:
#            line_i.append(name[1])  # not that townToMrt is not a array
#    print("line_{}= {} ".format(i,line_i))
#    print()
##############################################

# Gathering data for town, distance and price
distPrice=list(zip(town,rmType,DistToMrt,sellingPrice))
#print('distPrice', distPrice)
#distPrice=set(distPrice)
fig = plt.gcf()
for town in labels: 
    dist_town=[]
    price_town = []
    for name in distPrice:
        if name[0]== town and name[1]=='4-Room':
            dist_town.append(name[2])  # not that townToMrt is not a array
            price_town.append(name[3])
    #print("dist_{}= ({},{}) ".format(town,dist_town,price_town))
    
    # filter out towns that dont meet search criteria
    # to sort by dist
    if len(dist_town) > 0:
        tmp = list(zip(dist_town, price_town)) # zip dist and price together, so when sorting, they stay together
        tmp.sort() 
        tmpArray = np.array(tmp)
    
        dist_town_sorted = tmpArray[:,0]
        price_town_sorted = tmpArray[:,1]
    
    
        #print('price_town_sorted', price_town_sorted)
        #print('dist_town_sorted', dist_town_sorted)
        

# make subplots from my above plots
# We can ask for ALL THE AXES and put them into axes
fig, axes = plt.subplots(nrows=5, ncols=5, sharex=False, sharey=False, figsize=(15,15))
axes_list = [item for sublist in axes for item in sublist] 
axes_list_forHover = [item for sublist in axes for item in sublist] 
annot_list_forHover = []


for town in labels: 
    dist_town=[]
    price_town = []
    for name in distPrice:
        if name[0]== town and name[1]=='4-Room':
            dist_town.append(name[2])  # not that townToMrt is not a array
            price_town.append(name[3])
    #print("dist_{}= ({},{}) ".format(town,dist_town,price_town))
    
    # filter out towns that dont meet search criteria
    # to sort by dist
    dist_town_sorted = []
    price_town_sorted = []

    if len(dist_town) > 0:
        tmp = list(zip(dist_town, price_town)) # zip dist and price together, so when sorting, they stay together
        tmp.sort() 
        tmpArray = np.array(tmp)
    
        dist_town_sorted = tmpArray[:,0]
        price_town_sorted = tmpArray[:,1]
        
        
    # trying to generate correlation coeffient for each subgraph   
    np.random.seed(1)
    x = dist_town_sorted
    y = price_town_sorted
    coeff = np.corrcoef(x, y)
    #print("Correlation coefficient for {} : {}".format(town, coeff[0,1]))
    
    # Marker for floor level
    # many bubble plots are ".". each of these "." are considered a line
    ax = axes_list.pop(0)
    marker = list(zip(dist_town_sorted, price_town_sorted, floorlevel))
    # plot per graph per marker(point)
    for dist,price,unitLevel in marker:
        ax.plot(dist,price,".",clip_on=False, markersize=unitLevel, markeredgewidth=1.0,markeredgecolor=(0, 0, 0, 1),color="pink")
        
    
    #ax.plot(dist_town_sorted, price_town_sorted ,'.',clip_on=False)
    #ax.plot([0,1,2] )
    ax.set_title(town)
    ax.tick_params(
        which='both',
        bottom=False,
        left=False,
        right=False,
        top=False
    )
    ax.grid(linewidth=0.25)
    #ax.set_xlim((1950, 2020))
    #ax.set_xticks(range(0,2800, 1000))
    ax.spines['left'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.grid(linewidth=0.25)
    #ax.set_xlim((0,3000))
    ax.set_xlabel("Distance to Mrt")
    ax.set_ylabel("Selling Price")
    #wspace = 1
    #hspace =1
    fig.subplots_adjust(hspace=10)
    fig.suptitle('4-Room Flats Selling Price against Distance from MRT',fontsize =20)
    
    # Best fit line
    x = dist_town_sorted
    y = price_town_sorted 
    def best_fit_slope_and_intercept(x,y):
        m = (((np.mean(x)*np.mean(y)) - np.mean(x*y)) /
             ((np.mean(x)*np.mean(x)) - np.mean(x*x)))
        b = np.mean(y) - m*np.mean(x)

        return m, b

    if len(x) > 1 :
        m, b = best_fit_slope_and_intercept(x,y)
        regression_line = [(m*x_bar)+b for x_bar in x]
        #style.use('ggplot')
        ax.plot(x, regression_line, color='blue') # specific to each subplot
    
    
    # annotate on diagram
    annot = ax.annotate("", xy=(0,0), xytext=(20,20),textcoords="offset points",
                    bbox=dict(boxstyle="round", fc="w"),
                    arrowprops=dict(arrowstyle="->")) #xytext=(20,20) means 20,20 to the top and right of current point
    # xy = (0,0) is dummy. annot.xy is where you can change it
    fig.texts.append(ax.texts.pop())

    annot.set_visible(False) # until later will set visible

    
    annot_list_forHover.append(annot) # is an object, the thing that they display on the output is not so useful
    #print("annot_list_forHover",annot_list_forHover)
    

    
for ax in axes_list:
        ax.remove()
    #ax.spines['left'].set_visible(False)
    #ax.spines['top'].set_visible(False)
    #ax.spines['right'].set_visible(False)

plt.tight_layout()  
fig.subplots_adjust(top=0.91)


def clickoff():
    for item in annot_list_forHover:
        item.set_visible(False)
    fig.canvas.draw_idle()


# TO update annotation
def update_annot(subplotIdx, pointX, pointY, text):  
    print('update_annot',subplotIdx, pointX, pointY, text) 

    annot = annot_list_forHover[subplotIdx] 
    annot.xy = (pointX, pointY) # coordinates to put annotation box

    annot.xyann  =(-20,-20)
    print(dir(annot))
    print(annot.xy, annot.xyann, annot.xycoords)
    annot.set_visible(True)
    annot.set_text(text)
    annot.get_bbox_patch().set_facecolor("lightyellow")
    annot.get_bbox_patch().set_animated(True)
    # annot.get_bbox_patch().set_alpha(0.4)

# onclick event
def hover_callback(event):
    #print('hover_callback, event: ', event)
    #vis = annot.get_visible() 
    subplotIdx = axes_list_forHover.index(event.inaxes) 
    #print('ada: ', subplotIdx)
    if event.inaxes in axes_list_forHover: # assume unique
        #print(dir(event.inaxes))
        children = event.inaxes.get_children()  # what ever information in the subplots
        #print(children)
        blkAppend = "This is the blk number :  \n"
        toDisplayAnnot = False
        for child in children:
            cont, ind = child.contains(event)
            if cont ==True and isinstance(child,matplotlib.lines.Line2D) and child.get_marker()==".": 
                toDisplayAnnot = True # to display annot as long as one child matches
                x_data = child.get_xdata()[0]
                y_data = child.get_ydata()[0]
                
                subsetprice = unitDetails[unitDetails['sellingPrice'] == y_data]
                subsetdistFrMrt = subsetprice[subsetprice['DistToMrt'] == x_data]
                toprintBlk = "{}  \n".format(subsetdistFrMrt["blk"][0])
                blkAppend = blkAppend + toprintBlk 
                
        print(blkAppend)
        clickoff() # put it outside is because we can click anywhere can turn off the bbox.
        
        # true only if we click on the marker.
        if toDisplayAnnot:            
            update_annot(subplotIdx, x_data, y_data, blkAppend)  
            fig.canvas.draw_idle()
        
                #print(subsetprice)
                #print()
                #print(subsetdistFrMrt)
            
                
                # inverse to find the blk number
                
                #print(child.get_xdata()[0],",",child.get_ydata()[0])
                #print('child',child.get_marker(), cont, ind)

    #    for flier in bp["fliers"]:
    #        cont, ind = flier.contains(event)  #fliers is outliers 
    #        if cont:
    #            update_annot(flier, ind)
    #            annot.set_visible(True)
    #            fig.canvas.draw_idle()
    #        else:
    #            if vis:
    #           annot.set_visible(False)
    #                fig.canvas.draw_idle()




#fig.canvas.mpl_connect("motion_notify_event", hover_callback) # on hover
fig.canvas.mpl_connect("button_press_event", hover_callback) # on click

plt.show()    
print()
    
        
        #plt.plot(dist_town_sorted, price_town_sorted,label=town)
        #plt.legend()  # to make legend come out
        #print()

#################################################
# plotting

#print(distPrice[0][0])
#print(distPrice[0][1])
#print(distPrice)    
    


## format

#line_1 = np.random.randint(1,20,20)
#line_2 = np.random.randint(1,20,20)
#plt.plot(line_1,label="Line 1")
#plt.plot(line_2,label="Line 2")

#legend = plt.legend()
#plt.show()


In [1]:
#%matplotlib inline  # inline is a picture only.

import matplotlib
#matplotlib.use("WxAgg")
matplotlib.use("TkAgg")  #for pop up window
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import os
import seaborn as sns

os.getcwd()
# TO get a boxplot to see the number of 4-Rooms flats available for Nov 2018 Sale of balance flats

unitDetails = np.genfromtxt("Merged_unitsDetails_DistToMrt.csv", delimiter=",",skip_header=1,
                dtype=[('town','U24'),('rmType','U7'),('blk','U6'),('unitNum','U10'),
                      ('level','i4'),('sqm','i8'),
                      ('sellingPrice','i8'),('color','U12') ,('repurchasedFlat','U30'),('leaseLessThan60Yrs','U10'),
                      ('street','U30'),('probableCompletionDate','U10'),('deliveryPossessionDate','U30'),
                      ('leaseCommencementDate','U15'),('availableEthnicQuotaMl','U20'),
                      ('availableEthnicQuotaCh','U25'),('availableEthnicQuotaIn','U20'),('lastUpdate','U60'),
                      ('NearestMrt','U30'),('DistToMrt','f8'),('blkX','i8'),('blkY','i8')],
                missing_values=['na','-',''],filling_values=0, comments='!')

labels = ['Ang Mo Kio','Bedok','Bukit Batok','Bukit Merah','Bukit Panjang','Choa Chu Kang',
                'Clementi','Geylang','Hougang','Jurong East','Jurong West','Kallang / Whampoa','Pasir Ris','Punggol',
                'Queenstown','Sembawang','Sengkang','Serangoon','Tampines','Toa Payoh', 'Woodlands','Yishun']
# defining names
DistToMrt= unitDetails['DistToMrt']
town = unitDetails['town']
rmType= unitDetails['rmType']
sellingPrice=unitDetails['sellingPrice']
floorlevel = unitDetails['level']

#print(town)
#print(DistToMrt)
townToMrt= zip(town,DistToMrt)
townToMrt =set(townToMrt)

# Gathering data for each line on distance
#for i in labels:
#    line_i=[]
#    for name in townToMrt:
#        if name[0]== i:
#            line_i.append(name[1])  # not that townToMrt is not a array
#    print("line_{}= {} ".format(i,line_i))
#    print()
##############################################

# Gathering data for town, distance and price
distPrice=list(zip(town,rmType,DistToMrt,sellingPrice))
#print('distPrice', distPrice)
#distPrice=set(distPrice)
fig = plt.gcf()
for town in labels: 
    dist_town=[]
    price_town = []
    for name in distPrice:
        if name[0]== town and name[1]=='5-Room':
            dist_town.append(name[2])  # not that townToMrt is not a array
            price_town.append(name[3])
    #print("dist_{}= ({},{}) ".format(town,dist_town,price_town))
    
    # filter out towns that dont meet search criteria
    # to sort by dist
    if len(dist_town) > 0:
        tmp = list(zip(dist_town, price_town)) # zip dist and price together, so when sorting, they stay together
        tmp.sort() 
        tmpArray = np.array(tmp)
    
        dist_town_sorted = tmpArray[:,0]
        price_town_sorted = tmpArray[:,1]
    
    
        #print('price_town_sorted', price_town_sorted)
        #print('dist_town_sorted', dist_town_sorted)
        

# make subplots from my above plots
# We can ask for ALL THE AXES and put them into axes
fig, axes = plt.subplots(nrows=5, ncols=5, sharex=False, sharey=False, figsize=(15,15))
axes_list = [item for sublist in axes for item in sublist] 
axes_list_forHover = [item for sublist in axes for item in sublist] 
annot_list_forHover = []


for town in labels: 
    dist_town=[]
    price_town = []
    for name in distPrice:
        if name[0]== town and name[1]=='4-Room':
            dist_town.append(name[2])  # not that townToMrt is not a array
            price_town.append(name[3])
    #print("dist_{}= ({},{}) ".format(town,dist_town,price_town))
    
    # filter out towns that dont meet search criteria
    # to sort by dist
    dist_town_sorted = []
    price_town_sorted = []

    if len(dist_town) > 0:
        tmp = list(zip(dist_town, price_town)) # zip dist and price together, so when sorting, they stay together
        tmp.sort() 
        tmpArray = np.array(tmp)
    
        dist_town_sorted = tmpArray[:,0]
        price_town_sorted = tmpArray[:,1]
        
        
    # trying to generate correlation coeffient for each subgraph   
    np.random.seed(1)
    x = dist_town_sorted
    y = price_town_sorted
    coeff = np.corrcoef(x, y)
    #print("Correlation coefficient for {} : {}".format(town, coeff[0,1]))
    
    # Marker for floor level
    # many bubble plots are ".". each of these "." are considered a line
    ax = axes_list.pop(0)
    marker = list(zip(dist_town_sorted, price_town_sorted, floorlevel))
    # plot per graph per marker(point)
    for dist,price,unitLevel in marker:
        ax.plot(dist,price,".",clip_on=False, markersize=unitLevel, markeredgewidth=1.0,markeredgecolor=(0, 0, 0, 1),color="pink")
        
    
    #ax.plot(dist_town_sorted, price_town_sorted ,'.',clip_on=False)
    #ax.plot([0,1,2] )
    ax.set_title(town)
    ax.tick_params(
        which='both',
        bottom=False,
        left=False,
        right=False,
        top=False
    )
    ax.grid(linewidth=0.25)
    #ax.set_xlim((1950, 2020))
    #ax.set_xticks(range(0,2800, 1000))
    ax.spines['left'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.grid(linewidth=0.25)
    #ax.set_xlim((0,3000))
    ax.set_xlabel("Distance to Mrt")
    ax.set_ylabel("Selling Price")
    #wspace = 1
    #hspace =1
    fig.suptitle('4-Room Flats Selling Price against Distance from MRT',fontsize =20)
    
    # Best fit line
    x = dist_town_sorted
    y = price_town_sorted 
    def best_fit_slope_and_intercept(x,y):
        m = (((np.mean(x)*np.mean(y)) - np.mean(x*y)) /
             ((np.mean(x)*np.mean(x)) - np.mean(x*x)))
        b = np.mean(y) - m*np.mean(x)

        return m, b

    if len(x) > 1 :
        m, b = best_fit_slope_and_intercept(x,y)
        regression_line = [(m*x_bar)+b for x_bar in x]
        #style.use('ggplot')
        ax.plot(x, regression_line, color='blue') # specific to each subplot
    
    
    # annotate on diagram
    annot = ax.annotate("", xy=(0,0), xytext=(20,20),textcoords="offset points",
                    bbox=dict(boxstyle="round", fc="w"),
                    arrowprops=dict(arrowstyle="->")) #xytext=(20,20) means 20,20 to the top and right of current point
    # xy = (0,0) is dummy. annot.xy is where you can change it
    fig.texts.append(ax.texts.pop())

    annot.set_visible(False) # until later will set visible

    
    annot_list_forHover.append(annot) # is an object, the thing that they display on the output is not so useful
    #print("annot_list_forHover",annot_list_forHover)
    

    
for ax in axes_list:
        ax.remove()
    #ax.spines['left'].set_visible(False)
    #ax.spines['top'].set_visible(False)
    #ax.spines['right'].set_visible(False)

plt.tight_layout()  
fig.subplots_adjust(top=0.91)


def clickoff():
    for item in annot_list_forHover:
        item.set_visible(False)
    fig.canvas.draw_idle()


# TO update annotation
def update_annot(subplotIdx, pointX, pointY, text):  
    print('update_annot',subplotIdx, pointX, pointY, text) 

    annot = annot_list_forHover[subplotIdx] 
    annot.xy = (pointX, pointY) # coordinates to put annotation box

    annot.xyann  =(-20,-20)
    print(dir(annot))
    print(annot.xy, annot.xyann, annot.xycoords)
    annot.set_visible(True)
    annot.set_text(text)
    annot.get_bbox_patch().set_facecolor("lightyellow")
    annot.get_bbox_patch().set_animated(True)
    # annot.get_bbox_patch().set_alpha(0.4)

# onclick event
def hover_callback(event):
    #print('hover_callback, event: ', event)
    #vis = annot.get_visible() 
    subplotIdx = axes_list_forHover.index(event.inaxes) 
    #print('ada: ', subplotIdx)
    if event.inaxes in axes_list_forHover: # assume unique
        #print(dir(event.inaxes))
        children = event.inaxes.get_children()  # what ever information in the subplots
        #print(children)
        blkAppend = "This is the blk number :  \n"
        toDisplayAnnot = False
        for child in children:
            cont, ind = child.contains(event)
            if cont ==True and isinstance(child,matplotlib.lines.Line2D) and child.get_marker()==".": 
                toDisplayAnnot = True # to display annot as long as one child matches
                x_data = child.get_xdata()[0]
                y_data = child.get_ydata()[0]
                
                subsetprice = unitDetails[unitDetails['sellingPrice'] == y_data]
                subsetdistFrMrt = subsetprice[subsetprice['DistToMrt'] == x_data]
                toprintBlk = "{}  \n".format(subsetdistFrMrt["blk"][0])
                blkAppend = blkAppend + toprintBlk 
                
        print(blkAppend)
        clickoff() # put it outside is because we can click anywhere can turn off the bbox.
        
        # true only if we click on the marker.
        if toDisplayAnnot:            
            update_annot(subplotIdx, x_data, y_data, blkAppend)  
            fig.canvas.draw_idle()
        
                #print(subsetprice)
                #print()
                #print(subsetdistFrMrt)
            
                
                # inverse to find the blk number
                
                #print(child.get_xdata()[0],",",child.get_ydata()[0])
                #print('child',child.get_marker(), cont, ind)

    #    for flier in bp["fliers"]:
    #        cont, ind = flier.contains(event)  #fliers is outliers 
    #        if cont:
    #            update_annot(flier, ind)
    #            annot.set_visible(True)
    #            fig.canvas.draw_idle()
    #        else:
    #            if vis:
    #           annot.set_visible(False)
    #                fig.canvas.draw_idle()




#fig.canvas.mpl_connect("motion_notify_event", hover_callback) # on hover
fig.canvas.mpl_connect("button_press_event", hover_callback) # on click

plt.show()    
print()
    
        
        #plt.plot(dist_town_sorted, price_town_sorted,label=town)
        #plt.legend()  # to make legend come out
        #print()

#################################################
# plotting

#print(distPrice[0][0])
#print(distPrice[0][1])
#print(distPrice)    
    


## format

#line_1 = np.random.randint(1,20,20)
#line_2 = np.random.randint(1,20,20)
#plt.plot(line_1,label="Line 1")
#plt.plot(line_2,label="Line 2")

#legend = plt.legend()
#plt.show()


C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:356: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:78: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2392: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2326: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2326: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2400: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\